# TL; DR

- R's `data.table` is much faster than Python `pandas`' `DataFrame`: between 1.7 and 34 times for a set of standard operations.
- Note in particular the order of magnitude for conditional subsetting.
- In addition, although this notebook doesn't show it, `data.table` is also more memory efficient.
- Notes:
    - The benchmarking below is not super rigorous, but good enough to demonstrate the large differences between the two implementations.
    - All benchmarks are run on an average laptop under Windows 10, but these differences probably hold across OS and hardware.
    - The CSV write is astonishing. The test was run on a Samsung MZVLB512HBJQ which is an SSD drive, but maybe other hardware has better caching capabilities.

# Setup

We use `rpy2` to run a dual Python/R notebook. Using R, we create a dataframe and save it to disk as a `CSV` file, so that both implementations can retrieve and work on the same data.

In [1]:
import pandas as pd
import numpy as np
import timeit
import os

%load_ext rpy2.ipython

c:\users\leonardosilvestri\miniconda3\envs\py38\lib\site-packages\rpy2\robjects\packages.py:366: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


In [2]:
path = "~/Documents/tmp"

In [3]:
%%R
library(data.table)
library(rbenchmark)

R[write to console]: data.table 1.13.2 using 4 threads (see ?getDTthreads).  Latest news: r-datatable.com



In [4]:
%%R
path <- "~/tmp"

N = 1e7  # the total number of rows

a_h <- paste0("a", runif(2))
b_h <- paste0("b", runif(4))
c_h <- paste0("c", runif(1e2))
d_h <- paste0("d", runif(1e5))

dt <- data.table(a=rep(a_h, length.out=N),
                 b=rep(b_h, length.out=N),
                 c=rep(c_h, length.out=N),
                 d=rep(d_h, length.out=N),
                 f=runif(N),
                 g=runif(N))
                 
fwrite(dt, file.path(path, "temp.csv"))

In [5]:
%%R
dt_sample_d <- dt[!duplicated(dt[, .(a,b,c,d)])]
setnames(dt_sample_d, c("f","g"), c("ff", "gg"))
fwrite(dt_sample_d, file.path(path, "temp_sample_d.csv"))

In [6]:
%%R
print(nrow(dt_sample_d))

[1] 100000


# CSV read/write

## read

In [7]:
%%R -o csv_read_r

s <- Sys.time()
for (i in 1:3) 
    fread(file.path(path, "temp.csv"))
e <- Sys.time()
csv_read_r <- difftime(e, s, units="secs")

R[write to console]: |--------------------------------------------------|
|
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wr

In [8]:
csv_read_r.item()

17.232918977737427

In [9]:
s = timeit.default_timer()
for i in range(0, 3):
    pd.read_csv(os.path.join(path, "temp.csv"))   
e = timeit.default_timer()
csv_read_py = e-s

In [10]:
csv_read_py

39.2365018

## Write

In [11]:
%%R
print(dim(dt))

[1] 10000000        6


In [12]:
%%R -o csv_write_r

s <- Sys.time()
for (i in 1:3) 
    fwrite(dt, file.path(path, "temp.csv"))
e <- Sys.time()
csv_write_r <- difftime(e, s, units="secs")

In [13]:
csv_write_r.item()

5.207581996917725

In [14]:
df = pd.read_csv(os.path.join(path, "temp.csv"))
df.shape

(10000000, 6)

In [15]:
s = timeit.default_timer()
for i in range(0, 3):
    df.to_csv(os.path.join(path, "temp_a.csv"))   
e = timeit.default_timer()
csv_write_py = e-s

In [16]:
csv_write_py

178.4642288

# Subsetting

## Boolean condition

In [17]:
%%R -o subset_r
afind = dt$a[1]
bfind = dt$b[1]

s <- Sys.time()
for (i in 1:10)
    dt[a==afind & b==bfind]
e <- Sys.time()
subset_r <- difftime(e, s, units="secs")

In [18]:
subset_r.item()

1.6610119342803955

In [19]:
afind = df.a[0]
bfind = df.b[0]

s = timeit.default_timer()
for i in range(0, 10):
    df[(df.a==afind) & (df.b==bfind)]
e = timeit.default_timer()
subset_py = e-s

In [20]:
subset_py

16.302471100000048

## Integer index

In [21]:
%%R -o subset_int_r
idx <- seq(1, nrow(dt), by=2)

s <- Sys.time()
for (i in 1:10) 
    dt[idx]
e <- Sys.time()
subset_int_r <- difftime(e, s, units="secs")

In [22]:
subset_int_r

array([2.38238597])

In [23]:
idx = np.arange(0, df.shape[0], step=2)
s = timeit.default_timer()
for i in range(0,10):
    df.iloc[idx]
e = timeit.default_timer()
subset_int_py = e-s

In [24]:
subset_int_py

4.165189699999985

# Merge


In [25]:
%%R -o merge_r

s <- Sys.time()
for (i in 1:10)
    merge(dt, dt_sample_d, by=c("a","b","c","d"))
e <- Sys.time()
merge_r <- subset_r <- difftime(e, s, units="secs")

In [26]:
merge_r.item()

26.371317863464355

In [27]:
df_sample_d = pd.read_csv(os.path.join(path, "temp_sample_d.csv"))

s = timeit.default_timer()
for i in range(0,10):
    df.merge(df_sample_d, on=["a","b","c","d"])
e = timeit.default_timer()
merge_py = e-s

In [28]:
merge_py

51.16356560000003

# Groupby

## Built-in `mean`

In [29]:
%%R -o groupby_builtin_r

s <- Sys.time()
for (i in 1:10) 
    dt[, list(mean(f), mean(g)), by=.(a,b,c,d)]
e <- Sys.time()
groupby_builtin_r <- difftime(e, s, units="secs")

In [30]:
groupby_builtin_r.item()

7.784824848175049

In [31]:
s = timeit.default_timer()
for i in range(0, 10):
    df.groupby(['a', 'b', 'c', 'd']).mean()
e = timeit.default_timer()
groupby_builtin_py = e-s

In [32]:
groupby_builtin_py

52.83777810000004

## Custom function

In [33]:
%%R -o groupby_custom_r

get_nrows <- function(x) nrow(x)

s <- Sys.time()
for (i in 1:10)
    dt[, list(N=get_nrows(.SD)), by=.(a,b,c,d)]
e <- Sys.time()
groupby_custom_r <- difftime(e, s, units="secs")

In [34]:
groupby_custom_r.item()

10.229142904281616

In [35]:
def get_nrows(x):
    return x.shape[0]

s = timeit.default_timer()
for i in range(0, 10):
    df.groupby(['a', 'b', 'c', 'd']).apply(get_nrows)
e = timeit.default_timer()
groupby_custom_py = e-s

In [36]:
groupby_custom_py

126.97294210000001

## Custom function multiple variables

In [37]:
%%R -o groupby_custom_r_2

get_nrows <- function(x) nrow(x)

s <- Sys.time()
for (i in 1:10)
    dt[, list(mean=mean(g), N=get_nrows(.SD)), by=.(a,b,c,d)]
e <- Sys.time()
groupby_custom_r_2 <- difftime(e, s, units="secs")

In [38]:
groupby_custom_r_2.item()

11.376850843429565

In [39]:
def get_nrows(x):
    return {"mean": np.mean(x.g), "N": x.shape[0]}

s = timeit.default_timer()
for i in range(0, 10):
    df.groupby(['a', 'b', 'c', 'd']).apply(get_nrows)
e = timeit.default_timer()
groupby_custom_py_2 = e-s

In [40]:
groupby_custom_py_2

240.48145209999996

# Unique

## Elements of a column

In [41]:
%%R -o unique_r

s <- Sys.time()
for (i in 1:10)
    unique(dt$c)
e <- Sys.time()
unique_r <- difftime(e, s, units="secs")

In [42]:
unique_r.item()

1.4237329959869385

In [43]:
s = timeit.default_timer()
for i in range(0, 10):
    df.c.unique()
e = timeit.default_timer()
unique_py = e-s

In [44]:
unique_py

6.891304900000023

## Rows

In [45]:
%%R -o unique_row_r

s <- Sys.time()
for (i in 1:10)
        unique(dt, by=c("a","b","c","d"))
e <- Sys.time()
unique_row_r <- difftime(e, s, units="secs")

In [46]:
unique_row_r.item()

6.459529161453247

In [47]:
s = timeit.default_timer()
for i in range(0, 10):
    df.drop_duplicates(subset=["a","b","c","d"])
e = timeit.default_timer()
unique_row_py = e-s

In [48]:
unique_row_py

46.45240439999998

# Summary

In [49]:
summary = pd.DataFrame({"Operation": ["csv read", "csv write", 
                                      "bool subset", "int subset", 
                                      "merge",
                                      "groupby builtin", "groupby custom", "groupby custom multiple",
                                      "unique within col", "unique rows"], 
                 "pandas": [csv_read_py, csv_write_py, 
                            subset_py, subset_int_py,
                            merge_py,
                            groupby_builtin_py, groupby_custom_py, groupby_custom_py_2,
                            unique_py, unique_row_py], 
                 "data.table": [csv_read_r, csv_write_r, 
                                subset_r, subset_int_r,
                                merge_r,
                                groupby_builtin_r, groupby_custom_r, groupby_custom_r_2, 
                                unique_r, unique_row_r]})

In [50]:
summary["data.table"] = summary["data.table"].apply(lambda x: x.item())
summary["speedup"] = summary.pandas / summary["data.table"]

In [51]:
# times are in seconds:
summary

,Operation,pandas,data.table,speedup
0,csv read,39.236502,17.232919,2.276834
1,csv write,178.464229,5.207582,34.270076
2,bool subset,16.302471,1.661012,9.814783
3,int subset,4.165190,2.382386,1.748327
4,merge,51.163566,26.371318,1.940122
5,groupby builtin,52.837778,7.784825,6.787279
6,groupby custom,126.972942,10.229143,12.412862
7,groupby custom multiple,240.481452,11.376851,21.137787
8,unique within col,6.891305,1.423733,4.840307
9,unique rows,46.452404,6.459529,7.191299
